In [3]:
!pip install bs4
!pip install lxml
!pip install geopy
!pip install geocoder
!conda install -c conda-forge folium=0.5.0
!pip install selenium
!pip install pdfcrowd
!pip install bokeh
!pip install imgkit
!conda install selenium geckodriver firefox -c conda-forge
!pip install geographiclib

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    pandas-1.0.3               |   py36h830a2c2_1        11.1 MB  conda-forge
    toolz-0.10.0               |             py_0          46 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                  

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import geocoder # import geocoder
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library
import numpy as np
from geopy.distance import distance
import os
import time
from selenium import webdriver
import subprocess
import bokeh
import imgkit
import pdfcrowd
import sys
from geographiclib.geodesic import Geodesic

First I will get the user's location data. This would probably be their address.

In [5]:
address = 'Downtown Vancouver, BC, CAN'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

geod = Geodesic.WGS84
# print("Your location is {} N,{} W".format(latitude, longitude))

Let's create a quick map of our location:

In [6]:
map = folium.Map(location=[latitude, longitude], zoom_start=13)    
map

Now I will use Foursquare to find the nearby venues. Foursquare is going to return a .json file which I will need to process.

In [7]:
CLIENT_ID = 'SI5QNDDUPAKHLF1BKSB1RFWBFIAIT14VFITE1AO120000KMU' # Foursquare ID
CLIENT_SECRET = '11XILRUXTTXFUP1H5MDUKT3TSFXT5FRJ4XAYHL2DQOSM0KPW' # Foursquare Secret
VERSION = '20200402' # Foursquare API version
LIMIT = 500
radius = 10000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
# url

results = requests.get(url).json()
# results

Now I need to process this json file by extracting the category of each venue

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


Once my venues are put into a dataframe, I can list the venue types for my user and then let them choose which types of venues they want to learn the locations of

In [44]:
# Get unique venue categories and sort. If user wants to choose their venues alphabetically
venue_selection = sorted(nearby_venues['categories'].unique())
venue_selection

['Art Gallery',
 'Bakery',
 'Beach',
 'Belgian Restaurant',
 'Bookstore',
 'Boxing Gym',
 'Brewery',
 'Café',
 'Cheese Shop',
 'Clothing Store',
 'Coffee Shop',
 'Convenience Store',
 'Cosmetics Shop',
 'Deli / Bodega',
 'Dessert Shop',
 'Dog Run',
 'Donut Shop',
 'French Restaurant',
 'Fried Chicken Joint',
 'Furniture / Home Store',
 'Gourmet Shop',
 'Grocery Store',
 'Gym',
 'Gym / Fitness Center',
 'Hockey Arena',
 'Hotel',
 'Ice Cream Shop',
 'Italian Restaurant',
 'Japanese Restaurant',
 'Lebanese Restaurant',
 'Lighthouse',
 'Lingerie Store',
 'Liquor Store',
 'Lounge',
 'Market',
 'Middle Eastern Restaurant',
 'Movie Theater',
 'New American Restaurant',
 'Outdoor Sculpture',
 'Park',
 'Pizza Place',
 'Plaza',
 'Poke Place',
 'Ramen Restaurant',
 'Restaurant',
 'Sandwich Place',
 'Science Museum',
 'Sculpture Garden',
 'Seafood Restaurant',
 'Soccer Stadium',
 'Stadium',
 'Steakhouse',
 'Sushi Restaurant',
 'Tech Startup',
 'Trail',
 'Vietnamese Restaurant',
 'Waterfront']

In [42]:
# Choosing venues by their abundance.
nearby_venues['categories'].value_counts().head()

Hotel             8
Park              7
Trail             5
Sandwich Place    4
Coffee Shop       4
Name: categories, dtype: int64

In [41]:
nearby_venues.head()

,name,categories,lat,lng
0,iLoveKickboxing.com Vancouver,Boxing Gym,49.283143,-123.100976
1,Revolver,Coffee Shop,49.283187,-123.109288
2,Birds & The Beets,Café,49.283556,-123.102940
3,Meat & Bread,Sandwich Place,49.282646,-123.109499
4,Invoke Media,Tech Startup,49.283547,-123.104078


The user chooses something and I produce a new dataframe with all the locations

In [49]:
desired_venue = ['Park'] # From user input
selected_venues = nearby_venues[nearby_venues['categories'].isin(desired_venue)]
selected_venues.head()

,name,categories,lat,lng
20,Creekside Park,Park,49.274641,-123.102701
38,Coal Harbour Park,Park,49.289936,-123.125002
40,Harbour Green Park,Park,49.290062,-123.121719
45,David Lam Park,Park,49.272467,-123.123866
63,George Wainborn Park,Park,49.272383,-123.129433


Now that I have my selected venue, I want to calculate the distance that I am from each one. I'll be fancy and use the distance function from geopy and apply that on my original dataframe. I will also find the direction to the place.

In [50]:
# Create distance column
selected_venues['distance (m)'] = selected_venues.apply(lambda row: distance([latitude, longitude], [row['lat'], row['lng']]).m, axis=1)
# Create direction column 
selected_venues['direction'] = selected_venues.apply(lambda row: geod.Inverse(latitude, longitude, row['lat'], row['lng'])['azi1'], axis=1)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


I can sort my venues by their distance

In [57]:
selected_venues.sort_values(['distance (m)'])

,name,categories,lat,lng,distance (m),direction
20,Creekside Park,Park,49.274641,-123.102701,949.797004,176.355356
40,Harbour Green Park,Park,49.290062,-123.121719,1529.553842,-59.888130
38,Coal Harbour Park,Park,49.289936,-123.125002,1734.179263,-64.249632
45,David Lam Park,Park,49.272467,-123.123866,1898.600137,-128.793281
63,George Wainborn Park,Park,49.272383,-123.129433,2233.822078,-122.452775
74,Charleson Park,Park,49.266880,-123.124681,2376.580273,-139.632874
95,Vanier Park,Park,49.275813,-123.142147,2926.290369,-106.208376


Now I want to plot each of the locations on a map. I will use the distance and directions that I calculated in order to plot arrows showing the direction, and lines showing the straight-line path to the location. This will serve as the solution to the student who is learning orienteering.

In [58]:
map = folium.Map(location=[latitude, longitude], zoom_start=14)

# add markers to map
for lat, lng, label, angle in zip(selected_venues['lat'], selected_venues['lng'], selected_venues['name'], selected_venues['direction']):
    label = folium.Popup(label, parse_html=True)
    points = ([latitude, longitude], [lat, lng])
    folium.RegularPolygonMarker( # marks venues
        [lat, lng], 
        fill_color='blue',
        popup=label,
        number_of_sides=3, 
        radius=6, 
        rotation=angle-90).add_to(map)
    folium.CircleMarker( # marks origin
        [latitude, longitude],
        radius=2,
        popup='origin',
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map) 
    folium.PolyLine( # connects origin to venues
        points, 
        color="blue", 
        weight=2.5, 
        opacity=0.5).add_to(map)
map

In [48]:
# for lat, lng, label, angle in zip(selected_venues['lat'], selected_venues['lng'], selected_venues['name']):
#     closeup = folium.Map(location=[lat, lng], zoom_start=17)
#     img_name = 'images/{}'.format(label)
#     label = folium.Popup(label, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(closeup)  
#     closeup.save(img_name)

    
# # closeup